> **SWIFT TRAQ** : Customer Lifetime Value Analysis

###### **Author**: *DS/DA* Ochieng Festus

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = r"/content/drive/My Drive/SwiftTraq/Portfolio/001_CLV/data/"

In [ ]:
def load_data(filepath, chunk_size=1000):
  chunks = pd.read_csv(filepath, chunksize=chunk_size, delimiter='\t')
  return pd.concat(chunks, ignore_index=True)

In [ ]:
df_customer = load_data(file_path+'Clean_DimCustomer.csv')
df_sales = load_data(file_path+'Clean_FactInternetSales.csv')
df_date = load_data(file_path+'Clean_DimDate.csv')
df_geography = load_data(file_path+'DimGeography.csv')

In [ ]:
df_customer.head(3)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11000,26,AW00011000,NaN,Jon,V,Yang,0,1971-10-06,M,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,NaN,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,NaN,Eugene,L,Huang,0,1976-05-10,S,...,Professional,Profesional,Cadre,0,1,2243 W St.,NaN,1 (11) 500 555-0110,2011-01-15,0-1 Miles
2,11002,31,AW00011002,NaN,Ruben,NaN,Torres,0,1971-02-09,M,...,Professional,Profesional,Cadre,1,1,5844 Linden Land,NaN,1 (11) 500 555-0184,2011-01-07,2-5 Miles


In [ ]:
df_sales.head(3)

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.27,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.99,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.99,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00.000,2011-01-10 00:00:00.000,2011-01-05 00:00:00.000


In [ ]:
df_date.head(3)

,DateKey,FullDateAlternateKey,DayNumberOfWeek,EnglishDayNameOfWeek,SpanishDayNameOfWeek,FrenchDayNameOfWeek,DayNumberOfMonth,DayNumberOfYear,WeekNumberOfYear,EnglishMonthName,SpanishMonthName,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester
0,20050101,2005-01-01,7,Saturday,Sábado,Samedi,1,1,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
1,20050102,2005-01-02,1,Sunday,Domingo,Dimanche,2,2,2,January,Enero,Janvier,1,1,2005,1,3,2005,2
2,20050103,2005-01-03,2,Monday,Lunes,Lundi,3,3,2,January,Enero,Janvier,1,1,2005,1,3,2005,2


In [ ]:
df_geography.head(3)

,GeographyKey,City,StateProvinceCode,StateProvinceName,CountryRegionCode,EnglishCountryRegionName,SpanishCountryRegionName,FrenchCountryRegionName,PostalCode,SalesTerritoryKey,IpAddressLocator
0,1,Alexandria,NSW,New South Wales,AU,Australia,Australia,Australie,2015,9,198.51.100.2
1,2,Coffs Harbour,NSW,New South Wales,AU,Australia,Australia,Australie,2450,9,198.51.100.3
2,3,Darlinghurst,NSW,New South Wales,AU,Australia,Australia,Australie,2010,9,198.51.100.4


In [ ]:
#df_sales.info()

In [ ]:
# convert dates to actual dates

df_sales['OrderDate'] = pd.to_datetime(df_sales['OrderDate'])
df_date['FullDateAlternateKey'] = pd.to_datetime(df_date['FullDateAlternateKey'])

#Full customer name
df_customer['FullName'] = df_customer['FirstName'] + ' '+ df_customer['LastName']

###### Merge sales with customer and date

In [ ]:
df = pd.merge(df_sales, df_customer, on='CustomerKey')
df = pd.merge(df, df_date, left_on='OrderDateKey', right_on='DateKey')

###### Aggregate sales by customer, compute CLV-related metrics



In [ ]:
df_clv = df.groupby('CustomerKey').agg(
    TotalRevenue = ('SalesAmount', 'sum'),
    TotalOrders = ('SalesOrderNumber', 'nunique'),
    AvgOrderValue=('SalesAmount', 'mean'),
    LastPurchaseDate=('OrderDate', 'max'),
    FirstPurchaseDate=('OrderDate', 'min')
).reset_index()

df_clv['CustomerLifetime'] = (df_clv['LastPurchaseDate'] - df_clv['FirstPurchaseDate']).dt.days
df_clv = pd.merge(df_clv, df_customer[['CustomerKey', 'GeographyKey', 'FullName']], on='CustomerKey')
df_clv = pd.merge(df_clv, df_geography[['GeographyKey', 'EnglishCountryRegionName','CountryRegionCode']], on='GeographyKey')

In [ ]:
df_clv.head(3)

,CustomerKey,TotalRevenue,TotalOrders,AvgOrderValue,LastPurchaseDate,FirstPurchaseDate,CustomerLifetime,GeographyKey,FullName,EnglishCountryRegionName,CountryRegionCode
0,11000,8248.99,3,1031.123750,2013-05-03,2011-01-19,835,26,Jon Yang,Australia,AU
1,11001,6383.88,3,580.352727,2013-12-10,2011-01-15,1060,37,Eugene Huang,Australia,AU
2,11002,8114.04,3,2028.510000,2013-02-23,2011-01-07,778,31,Ruben Torres,Australia,AU


In [ ]:
df_clv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CustomerKey               18484 non-null  int64         
 1   TotalRevenue              18484 non-null  float64       
 2   TotalOrders               18484 non-null  int64         
 3   AvgOrderValue             18484 non-null  float64       
 4   LastPurchaseDate          18484 non-null  datetime64[ns]
 5   FirstPurchaseDate         18484 non-null  datetime64[ns]
 6   CustomerLifetime          18484 non-null  int64         
 7   GeographyKey              18484 non-null  int64         
 8   FullName                  18484 non-null  object        
 9   EnglishCountryRegionName  18484 non-null  object        
 10  CountryRegionCode         18484 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(4), object(3)
memory usage: 1.6+ MB


In [ ]:
# convert customer key to string for px (categorical) plots

df_clv['CustomerKey'] = df_clv['CustomerKey'].astype(str)

#### **VISUALIZATIONS**

###### Distribution of customer Lifetime value

In [ ]:
fig1 = px.histogram(
    df_clv,
    x='CustomerLifetime',
    nbins=30,
    title='Distribution of Customer Lifetime (in days)',
    template='plotly_dark',
    color_discrete_sequence=['#6495ED'] ##636EFA
)
fig1.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    bargap=0.2
)
fig1.show()

###### Revenue Contribution by Region

In [ ]:
fig2 = px.bar(
    df_clv.groupby('EnglishCountryRegionName')['TotalRevenue'].sum().reset_index(),
    x='EnglishCountryRegionName',
    y='TotalRevenue',
    title='Revenue Contribution by Region',
    labels={'TotalRevenue': 'Total Revenue', 'EnglishCountryRegionName': 'Country'},
    template='plotly_dark',
    color='TotalRevenue',
    color_continuous_scale='blues'
)
fig2.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)
fig2.show()

###### Average Order Value Distribution

In [ ]:
fig3 = px.box(
    df_clv,
    y='AvgOrderValue',
    title='Distribution of Average Order Value',
    labels={'AvgOrderValue': ' Average Order Value'},
    template='plotly_dark',
    color_discrete_sequence=['#6495ED']
)
fig3.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)
fig3.show()

###### Total Revenue Vs. Total Orders

In [ ]:
fig4 = px.scatter(
    df_clv,
    x='TotalOrders',
    y='TotalRevenue',
    size='TotalRevenue',
    color='EnglishCountryRegionName',
    title='Total Revenue vs. Total Orders by Customer',
    labels={'TotalOrders': 'Total Orders', 'TotalRevenue': 'Total Revenue'},
    template='plotly_dark'
)
fig4.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    legend_title_text='Country/Region'
)
fig4.show()

###### Revenue Breakdown

In [ ]:
df_top_customers = df_clv.nlargest(10, 'TotalRevenue')
fig6 = px.bar(
    df_top_customers,
    x='FullName',
    y='TotalRevenue',
    title='Top 10 Customers by Revenue',
    labels={'FullName': 'Customer Name', 'TotalRevenue': 'Total Revenue'},
    template='plotly_dark',
    color='TotalRevenue',
    color_continuous_scale='Blues' #Bluered
)
fig6.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)
fig6.show()

###### @SWIFT TRAQ